In [13]:
import ipywidgets as widget
from IPython.display import display
from pymongo import MongoClient
import datetime

In [14]:
client=MongoClient()
db=client.hospital

In [59]:
class process_doctor(object):
    
    def __init__(self):
        pass
    
    def add_to_db(self,user_name,user_role):
        users=db.users
        if len(user_name)>=3:
            users.insert_one({'name':user_name,'role':user_role})
            print "Added!"
        else:
            print "Invalid name!"
    
    def get_all(self):
        users=db.users
        all_doctors=[]
        for record in users.find({'role': 1,'links_to':{'$exists':False}},{'name':1,'_id':0}):
            all_doctors.append(str(record.get("name")))
        return all_doctors
    
    def link_to_patient(self,u_doctor,u_patient):
        users=db.users
        users.update_one({'name':u_doctor},{'$set':{'links_to':u_patient}})
        print "Linked!"
            
class process_patient(object):
    
    def __init__(self):
        pass
    
    def add_to_db(self,user_name,user_role):
        users=db.users
        if len(user_name)>=3:
            users.insert_one({'name':user_name,'role':user_role})
            print "Added!"
        else:
            print "Invalid name!" 
    
    def get_all(self):
        users=db.users
        all_patients={}
        for record in users.find({"role": 2},{'name':1}):
            all_patients.update({str(record.get("name")):record.get("_id")})
        return all_patients

In [44]:
doctor_field=widget.Text()
patient_field=widget.Text()

def render_patient_ui(patient_field):
    caption_1=widget.Label(value="Add a patient");
    display(caption_1,patient_field)
    patient_field.on_submit(register_patient)
        
def register_patient(value):
    patient=process_patient()
    patient.add_to_db(patient_field.value,2)

def render_doctor_ui(doctor_field):
    caption_2=widget.Label(value="Add a doctor");
    display(caption_2,doctor_field)
    doctor_field.on_submit(register_doctor)
        
def register_doctor(value):
    doctor=process_doctor()
    doctor.add_to_db(doctor_field.value,1)           

In [45]:
#add a doctor
render_doctor_ui(doctor_field)
#add a patient
render_patient_ui(patient_field)

Added!


In [67]:
#schedule appointment

def get_apt_details(value):
    add_apt=process_doctor()
    add_apt.link_to_patient(doctors_list.value,patients_list.value)
    
doctors=process_doctor()
all_doctors=doctors.get_all()
if len(all_doctors)>0:
    doctors_list = widget.Dropdown(
        options=all_doctors,
        description='Select Doctor:'
    )
    patients=process_patient()
    all_patients=patients.get_all()
    patients_list = widget.Dropdown(
        options=all_patients,
        description='Select Patient:'
    )
    display(doctors_list)
    display(patients_list)

    create_link=widget.Button(description='Add Appointment');
    display(create_link)

    create_link.on_click(get_apt_details)
else:
    print "No available docs, please add first!"

No available docs, please add first!
